<a href="https://colab.research.google.com/github/vitthal-bhandari/SMS-Spam-Detection/blob/master/spam_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import os
import io
tf.__version__

'2.7.0'

In [ ]:
# Download the zip file
path_to_zip = tf.keras.utils.get_file("smsspamcollection.zip", origin="https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip", extract=True)

# Unzip the file into a folder
!unzip $path_to_zip -d data

212992/203415 [===============================] - 0s 1us/step
Archive:  /root/.keras/datasets/smsspamcollection.zip
  inflating: data/SMSSpamCollection  
  inflating: data/readme             


In [ ]:
# Let's see if we read the data correctly
lines = io.open('data/SMSSpamCollection').read().strip().split('\n')
lines[0]

'ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [ ]:
spam_dataset = []
count = 0
for line in lines:
  label, text = line.split('\t')
  if label.lower().strip() == 'spam':
    spam_dataset.append((1, text.strip()))
    count += 1
  else:
    spam_dataset.append(((0, text.strip())))
print(spam_dataset[0])
print("Spam: ", count)

(0, 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...')
Spam:  747


In [ ]:
import pandas as pd

df = pd.DataFrame(spam_dataset, columns=['Spam', 'Message'])

In [ ]:
df.head()

,Spam,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
import re

def message_length(x):
  #returns total number of characters
  return len(x)

def num_capitals(x):
  _, count = re.subn(r'[A-Z]', '', x)
  return count

def num_punctuation(x):
  _, count = re.subn(r'\W', '', x)
  return count

In [ ]:
df['Capitals'] = df['Message'].apply(num_capitals)
df['Punctuation'] = df['Message'].apply(num_punctuation)
df['Length'] = df['Message'].apply(message_length)
df.describe()

,Spam,Capitals,Punctuation,Length
count,5574.000000,5574.000000,5574.000000,5574.000000
mean,0.134015,5.621636,18.942591,80.443488
std,0.340699,11.683233,14.825994,59.841746
min,0.000000,0.000000,0.000000,2.000000
25%,0.000000,1.000000,8.000000,36.000000
50%,0.000000,2.000000,15.000000,61.000000
75%,0.000000,4.000000,27.000000,122.000000
max,1.000000,129.000000,253.000000,910.000000


In [ ]:
train=df.sample(frac=0.8,random_state=42) #random state is a seed value
test=df.drop(train.index)
x_train = train[['Length', 'Capitals', 'Punctuation']]
y_train = train[['Spam']]
x_test = test[['Length', 'Capitals', 'Punctuation']]
y_test = test[['Spam']]

In [ ]:
train.describe()

,Spam,Capitals,Punctuation,Length
count,4459.000000,4459.000000,4459.000000,4459.000000
mean,0.132765,5.519399,18.886522,80.316439
std,0.339359,11.405424,14.602023,59.346407
min,0.000000,0.000000,0.000000,2.000000
25%,0.000000,1.000000,8.000000,35.000000
50%,0.000000,2.000000,15.000000,61.000000
75%,0.000000,4.000000,27.000000,122.000000
max,1.000000,129.000000,253.000000,910.000000


In [ ]:
test.describe()

,Spam,Capitals,Punctuation,Length
count,1115.000000,1115.000000,1115.000000,1115.000000
mean,0.139013,6.030493,19.166816,80.951570
std,0.346116,12.731059,15.694599,61.807655
min,0.000000,0.000000,0.000000,2.000000
25%,0.000000,1.000000,8.000000,36.000000
50%,0.000000,2.000000,15.000000,61.000000
75%,0.000000,4.000000,28.000000,123.000000
max,1.000000,127.000000,195.000000,790.000000


In [ ]:
# Basic 1-layer neural network model for evaluation
def make_model(input_dims=3, num_units=12):
  model=tf.keras.Sequential()
  # Adds a densely-connected layer with 12 units to the model:
  model.add(tf.keras.layers.Dense(num_units, input_dim=input_dims, activation='relu'))
  # Add a sigmoid layer with a binary output unit:
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
x_train

,Length,Capitals,Punctuation
3690,25,1,4
3527,161,107,48
724,40,1,7
3370,69,3,17
468,37,1,8
...,...,...,...
3280,444,44,114
3186,65,50,14
3953,81,2,23
2768,38,2,8


In [ ]:
model=make_model()
model.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
446/446 [==============================] - 4s 4ms/step - loss: 0.6184 - accuracy: 0.7681
Epoch 2/10
446/446 [==============================] - 2s 4ms/step - loss: 0.3594 - accuracy: 0.8948
Epoch 3/10
446/446 [==============================] - 2s 4ms/step - loss: 0.2971 - accuracy: 0.9103
Epoch 4/10
446/446 [==============================] - 2s 4ms/step - loss: 0.2575 - accuracy: 0.9197
Epoch 5/10
446/446 [==============================] - 2s 4ms/step - loss: 0.2277 - accuracy: 0.9233
Epoch 6/10
446/446 [==============================] - 2s 4ms/step - loss: 0.2067 - accuracy: 0.9303
Epoch 7/10
446/446 [==============================] - 2s 4ms/step - loss: 0.1946 - accuracy: 0.9343
Epoch 8/10
446/446 [==============================] - 2s 4ms/step - loss: 0.1847 - accuracy: 0.9381
Epoch 9/10
446/446 [==============================] - 2s 4ms/step - loss: 0.1837 - accuracy: 0.9390
Epoch 10/10
446/446 [==============================] - 2s 4ms/step - loss: 0.1813 - accuracy: 0.9374

In [ ]:
model.evaluate(x_test, y_test)

35/35 [==============================] - 0s 3ms/step - loss: 0.1969 - accuracy: 0.9345


[0.19691359996795654, 0.9345291256904602]

In [ ]:
import numpy as np
y_train_pred=(model.predict(x_train) > 0.5).astype("int32")
# y_train_pred = model.predict(x_train)
# y_train_pred=np.argmax(y_train_pred,axis=1)

In [ ]:
# confusion matrix
tf.math.confusion_matrix(tf.constant(y_train.Spam), y_train_pred)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3742,  125],
       [ 154,  438]], dtype=int32)>

In [ ]:
!pip install stanza

     |████████████████████████████████| 432 kB 5.3 MB/s 
     |████████████████████████████████| 170 kB 49.2 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=0ea25c51ed878663bf2de3d62aafbbcc1c4ee7a19e2163e137237a197f21a648
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [ ]:
import stanza

In [ ]:
en = stanza.download('en')

2021-12-25 14:50:31 INFO: Downloading default packages for language: en (English)...


2021-12-25 14:50:58 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
en = stanza.Pipeline(lang='en')

2021-12-25 14:50:58 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2021-12-25 14:50:58 INFO: Use device: gpu
2021-12-25 14:50:58 INFO: Loading: tokenize
2021-12-25 14:51:09 INFO: Loading: pos
2021-12-25 14:51:09 INFO: Loading: lemma
2021-12-25 14:51:09 INFO: Loading: depparse
2021-12-25 14:51:10 INFO: Loading: sentiment
2021-12-25 14:51:10 INFO: Loading: constituency
2021-12-25 14:51:11 INFO: Loading: ner
2021-12-25 14:51:11 INFO: Done loading processors!


In [ ]:
sentence = 'Go until jurong point, crazy.. Available only in bugis n great world'
sentence.split()

['Go',
 'until',
 'jurong',
 'point,',
 'crazy..',
 'Available',
 'only',
 'in',
 'bugis',
 'n',
 'great',
 'world']

In [ ]:
tokenized = en(sentence)

In [ ]:
len(tokenized.sentences)

2

In [ ]:
for snt in tokenized.sentences:
  for word in snt.tokens:
    print(word.text)
  print("<End of Sentence>")

Go
until
jurong
point
,
crazy
..
<End of Sentence>
Available
only
in
bugis
n
great
world
<End of Sentence>


In [ ]:
!pip install stopwordsiso

     |████████████████████████████████| 73 kB 1.4 MB/s 


In [ ]:
import stopwordsiso as stopwords

stopwords.langs()

{'af',
 'ar',
 'bg',
 'bn',
 'br',
 'ca',
 'cs',
 'da',
 'de',
 'el',
 'en',
 'eo',
 'es',
 'et',
 'eu',
 'fa',
 'fi',
 'fr',
 'ga',
 'gl',
 'gu',
 'ha',
 'he',
 'hi',
 'hr',
 'hu',
 'hy',
 'id',
 'it',
 'ja',
 'ko',
 'ku',
 'la',
 'lt',
 'lv',
 'mr',
 'ms',
 'nl',
 'no',
 'pl',
 'pt',
 'ro',
 'ru',
 'sk',
 'sl',
 'so',
 'st',
 'sv',
 'sw',
 'th',
 'tl',
 'tr',
 'uk',
 'ur',
 'vi',
 'yo',
 'zh',
 'zu'}

In [ ]:
sorted(stopwords.stopwords('en'))

["'ll",
 "'tis",
 "'twas",
 "'ve",
 '10',
 '39',
 'a',
 "a's",
 'able',
 'ableabout',
 'about',
 'above',
 'abroad',
 'abst',
 'accordance',
 'according',
 'accordingly',
 'across',
 'act',
 'actually',
 'ad',
 'added',
 'adj',
 'adopted',
 'ae',
 'af',
 'affected',
 'affecting',
 'affects',
 'after',
 'afterwards',
 'ag',
 'again',
 'against',
 'ago',
 'ah',
 'ahead',
 'ai',
 "ain't",
 'aint',
 'al',
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'alongside',
 'already',
 'also',
 'although',
 'always',
 'am',
 'amid',
 'amidst',
 'among',
 'amongst',
 'amoungst',
 'amount',
 'an',
 'and',
 'announce',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anymore',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'ao',
 'apart',
 'apparently',
 'appear',
 'appreciate',
 'appropriate',
 'approximately',
 'aq',
 'ar',
 'are',
 'area',
 'areas',
 'aren',
 "aren't",
 'arent',
 'arise',
 'around',
 'arpa',
 'as',
 'aside',
 'ask',
 'asked',
 'asking',
 'asks',
 'associated

In [ ]:
en = stanza.Pipeline(lang='en', processors='tokenize')
en_sw = stopwords.stopwords('en')
def word_counts(x, pipeline=en):
  doc = pipeline(x)
  count=0
  for sentence in doc.sentences:
    for token in sentence.tokens:
      if token.text.lower() not in en_sw:
        count += 1
  return count

2021-12-25 10:27:56 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2021-12-25 10:27:56 INFO: Use device: gpu
2021-12-25 10:27:56 INFO: Loading: tokenize
2021-12-25 10:27:56 INFO: Done loading processors!


In [ ]:
df['Words'] = df['Message'].apply(word_counts)

In [ ]:
train['Words'] = train['Message'].apply(word_counts)
test['Words'] = test['Message'].apply(word_counts)

In [ ]:
x_train = train[['Length', 'Punctuation', 'Capitals', 'Words']]
y_train = train[['Spam']]

x_test = test[['Length', 'Punctuation', 'Capitals' , 'Words']]
y_test = test[['Spam']]

model = make_model(input_dims=4)

In [ ]:
model.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
446/446 [==============================] - 2s 4ms/step - loss: 8.8298 - accuracy: 0.5584
Epoch 2/10
446/446 [==============================] - 2s 4ms/step - loss: 0.4030 - accuracy: 0.8681
Epoch 3/10
446/446 [==============================] - 2s 4ms/step - loss: 0.2992 - accuracy: 0.9040
Epoch 4/10
446/446 [==============================] - 2s 4ms/step - loss: 0.2591 - accuracy: 0.9184
Epoch 5/10
446/446 [==============================] - 2s 4ms/step - loss: 0.2414 - accuracy: 0.9253
Epoch 6/10
446/446 [==============================] - 2s 4ms/step - loss: 0.2324 - accuracy: 0.9246
Epoch 7/10
446/446 [==============================] - 2s 4ms/step - loss: 0.2246 - accuracy: 0.9220
Epoch 8/10
446/446 [==============================] - 2s 4ms/step - loss: 0.2245 - accuracy: 0.9215
Epoch 9/10
446/446 [==============================] - 2s 4ms/step - loss: 0.2248 - accuracy: 0.9208
Epoch 10/10
446/446 [==============================] - 2s 4ms/step - loss: 0.2194 - accuracy: 0.9237

In [ ]:
model.evaluate(x_test, y_test)

35/35 [==============================] - 0s 3ms/step - loss: 0.2426 - accuracy: 0.9130


[0.2425527572631836, 0.913004457950592]

In [ ]:
#after stopwords removal
train['Words'] = train['Message'].apply(word_counts)
test['Words'] = test['Message'].apply(word_counts)
x_train = train[['Length', 'Punctuation', 'Capitals', 'Words']]
y_train = train[['Spam']]
x_test = test[['Length', 'Punctuation', 'Capitals', 'Words']]
y_test = test[['Spam']]
model = make_model(input_dims=4)
model.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
446/446 [==============================] - 2s 4ms/step - loss: 1.8621 - accuracy: 0.7827
Epoch 2/10
446/446 [==============================] - 2s 4ms/step - loss: 0.5690 - accuracy: 0.8769
Epoch 3/10
446/446 [==============================] - 2s 4ms/step - loss: 0.3695 - accuracy: 0.8755
Epoch 4/10
446/446 [==============================] - 2s 4ms/step - loss: 0.3000 - accuracy: 0.8941
Epoch 5/10
446/446 [==============================] - 2s 4ms/step - loss: 0.2580 - accuracy: 0.9087
Epoch 6/10
446/446 [==============================] - 2s 4ms/step - loss: 0.2488 - accuracy: 0.9143
Epoch 7/10
446/446 [==============================] - 2s 4ms/step - loss: 0.2353 - accuracy: 0.9157
Epoch 8/10
446/446 [==============================] - 2s 4ms/step - loss: 0.2271 - accuracy: 0.9157
Epoch 9/10
446/446 [==============================] - 2s 4ms/step - loss: 0.2231 - accuracy: 0.9195
Epoch 10/10
446/446 [==============================] - 2s 4ms/step - loss: 0.2232 - accuracy: 0.9170

In [ ]:
model.evaluate(x_test, y_test)

35/35 [==============================] - 0s 3ms/step - loss: 0.2997 - accuracy: 0.8762


[0.2997250556945801, 0.8762331604957581]

In [ ]:
en = stanza.Pipeline(lang='en')

txt = "Yo you around? A friend of mine's lookin."
pos = en(txt)

2021-12-25 10:32:42 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2021-12-25 10:32:42 INFO: Use device: gpu
2021-12-25 10:32:42 INFO: Loading: tokenize
2021-12-25 10:32:42 INFO: Loading: pos
2021-12-25 10:32:42 INFO: Loading: lemma
2021-12-25 10:32:42 INFO: Loading: depparse
2021-12-25 10:32:43 INFO: Loading: sentiment
2021-12-25 10:32:43 INFO: Loading: constituency
2021-12-25 10:32:44 INFO: Loading: ner
2021-12-25 10:32:45 INFO: Done loading processors!


In [ ]:
def print_pos(doc):
    text = ""
    for sentence in doc.sentences:
        for token in sentence.tokens:
            text += token.words[0].text + "/" + token.words[0].upos + " "
        text += "\n"
    return text

In [ ]:
print(print_pos(pos))

Yo/PRON you/PRON around/ADV ?/PUNCT 
A/DET friend/NOUN of/ADP mine/PRON 's/AUX lookin/ADJ ./PUNCT 



In [ ]:
en = stanza.Pipeline(lang='en', processors='tokenize')
en_sw = stopwords.stopwords('en')

def word_counts_v3(x, pipeline=en):
  doc = pipeline(x)
  totals = 0.
  count = 0.
  non_word = 0.
  for sentence in doc.sentences:
    totals += len(sentence.tokens)  # (1)
    for token in sentence.tokens:
        if token.text.lower() not in en_sw:
          if token.words[0].upos not in ['PUNCT', 'SYM']:
            count += 1.
          else:
            non_word += 1.
  non_word = non_word / totals
  return pd.Series([count, non_word], index=['Words_NoPunct', 'Punct'])

2021-12-25 10:32:45 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2021-12-25 10:32:45 INFO: Use device: gpu
2021-12-25 10:32:45 INFO: Loading: tokenize
2021-12-25 10:32:45 INFO: Done loading processors!


In [ ]:
train_tmp = train['Message'].apply(word_counts_v3)
train = pd.concat([train, train_tmp], axis=1)
train.describe()

,Spam,Capitals,Punctuation,Length,Words,Words_NoPunct,Punct
count,4459.000000,4459.000000,4459.000000,4459.000000,4459.000000,4459.000000,4459.0
mean,0.132765,5.519399,18.886522,80.316439,9.213725,9.213725,0.0
std,0.339359,11.405424,14.602023,59.346407,8.003355,8.003355,0.0
min,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.0
25%,0.000000,1.000000,8.000000,35.000000,4.000000,4.000000,0.0
50%,0.000000,2.000000,15.000000,61.000000,7.000000,7.000000,0.0
75%,0.000000,4.000000,27.000000,122.000000,13.000000,13.000000,0.0
max,1.000000,129.000000,253.000000,910.000000,157.000000,157.000000,0.0


In [ ]:
test_tmp = test['Message'].apply(word_counts_v3)
test = pd.concat([test, test_tmp], axis=1)
test.describe()

,Spam,Capitals,Punctuation,Length,Words,Words_NoPunct,Punct
count,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.0
mean,0.139013,6.030493,19.166816,80.951570,9.498655,9.498655,0.0
std,0.346116,12.731059,15.694599,61.807655,8.212000,8.212000,0.0
min,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.0
25%,0.000000,1.000000,8.000000,36.000000,4.000000,4.000000,0.0
50%,0.000000,2.000000,15.000000,61.000000,7.000000,7.000000,0.0
75%,0.000000,4.000000,28.000000,123.000000,14.000000,14.000000,0.0
max,1.000000,127.000000,195.000000,790.000000,82.000000,82.000000,0.0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
corpus = [
"I like fruits. Fruits like bananas",
"I love bananas but eat an apple",
"An apple a day keeps the doctor away"
]

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()

array(['an', 'apple', 'away', 'bananas', 'but', 'day', 'doctor', 'eat',
       'fruits', 'keeps', 'like', 'love', 'the'], dtype=object)

In [ ]:
X.toarray()

array([[0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 2, 0, 0],
       [1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0],
       [1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1]])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(X.toarray())

array([[1.        , 0.13608276, 0.        ],
       [0.13608276, 1.        , 0.3086067 ],
       [0.        , 0.3086067 , 1.        ]])

In [ ]:
query = vectorizer.transform(["apple and bananas"])
cosine_similarity(X, query)

array([[0.23570226],
       [0.57735027],
       [0.26726124]])

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
tfidf = transformer.fit_transform(X.toarray())
pd.DataFrame(tfidf.toarray(),
columns=vectorizer.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,an,apple,away,bananas,but,day,doctor,eat,fruits,keeps,like,love,the
0,0.000000,0.000000,0.000000,0.230408,0.000000,0.000000,0.000000,0.000000,0.688081,0.000000,0.688081,0.000000,0.000000
1,0.321267,0.321267,0.000000,0.321267,0.479709,0.000000,0.000000,0.479709,0.000000,0.000000,0.000000,0.479709,0.000000
2,0.275785,0.275785,0.411797,0.000000,0.000000,0.411797,0.411797,0.000000,0.000000,0.411797,0.000000,0.000000,0.411797


In [ ]:
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
model_w2v = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded
